<a href="https://colab.research.google.com/github/nicoRomeroCuruchet/DynamicProgramming/blob/main/testing_bary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np
from tqdm import tqdm
import gymnasium as gym
from itertools import product
from scipy.spatial import KDTree
from scipy.optimize import minimize
from PolicyIteration import PolicyIteration
from classic_control.cartpole import CartPoleEnv 
from classic_control.continuous_mountain_car import Continuous_MountainCarEnv

In [ ]:
env=CartPoleEnv(sutton_barto_reward=True)
# position thresholds:
x_lim = 2.5#env.x_threshold  
theta_lim = 0.25#env.theta_threshold_radians 
# velocity thresholds:
x_dot_lim = 2.5
theta_dot_lim = 2.5

bins_space = {
    "x_space": np.linspace(-x_lim, x_lim, 12),                         # position space         (0)
    "x_dot_space": np.linspace(-x_dot_lim, x_dot_lim, 12),             # velocity space         (1)
    "theta_space": np.linspace(-theta_lim, theta_lim, 12),             # angle space            (2)
    "theta_dot_space": np.linspace(-theta_dot_lim, theta_dot_lim, 12), # angular velocity space (3)
}

In [3]:
pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=[0, 1],
    gamma=0.99,
    theta=1e-2
)

pi.run()

  0%|          | 0/100 [08:26<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
def get_optimal_action(state:np.array, optimal_policy:PolicyIteration):
    """
    Aproximate the optimal action for a given state using the provided optimal policy
    with barycentric interpolation.

    Parameters:
    state (np.array): The state for which to determine the optimal action.
    optimal_policy (PolicyIteration): The optimal policy used to determine the action.

    Returns:
    action: The optimal action for the given state.
    """
    _, neighbors  = optimal_policy.kd_tree.query([state], k=optimal_policy.num_simplex_points)
    simplex       = optimal_policy.points[neighbors[0]]
    lambdas       = optimal_policy.barycentric_coordinates(state, simplex)

    actions = optimal_policy.action_space
    probabilities = np.zeros(len(actions))

    for i, l in enumerate(lambdas):
        for j, action in enumerate(actions):
            if optimal_policy.policy[tuple(simplex[i])][action] > 0:
                probabilities[j] += l

    argmax = lambda x: max(enumerate(x), key=lambda x: x[1])[0]
    action = actions[argmax(probabilities)]

    return action

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

num_episodes = 10000
task = CartPoleEnv(render_mode="human") # Continuous_MountainCarEnv(render_mode="human")  | CartPoleEnv(render_mode="human")
max_velocity = -1
max_theta_dot = -1
for episode in range(0, num_episodes):
    observation, _ = task.reset()
    total_reward = 0
    for timestep in range(1, 1000):
        action = get_optimal_action(observation, pi)
        observation, reward, terminated, done, info = task.step(action)
        total_reward += reward
        max_velocity = max(max_velocity, abs(observation[1]))
        max_theta_dot = max(max_theta_dot, abs(observation[3]))
        if terminated:
            print(f"Episode {episode} finished after {timestep} timesteps")
            print(f"Total reward: {total_reward}")
            print(f"Max velocity: {max_velocity}")
            print(f"Max theta_dot: {max_theta_dot}")
            break

In [ ]:
#requirements:
#numpy
#tqdm 
#gymnasium
#scipy
#loguru